<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW2_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4%2BAI%E5%9B%9E%E9%A5%8B(%E7%84%A1Gradio).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **操作說明**
  有三種動作可供選擇



---



##   **新增資料**

  ***日期時間自動輸入***
  1.  輸入項目（1: 早餐, 2: 午餐, 3: 晚餐, 4: 飲料, 5: 交通, 6: 就學用品, 7: 日常用品）
  2.  輸入品項
  3.  輸入金額
  4.  是否要AA分攤（yes or no）

          yes：自動除以輸入之人數並顯示實付金額在輸出區與試算表


---


##   **刪除資料**

依照提供之表格輸入要刪除的資料列號


---


##   **統整資料**

Gemini的分析回饋

    ***由於Gradio的時間限制 or Gemini API的問題，成功跑出結果的僅有無Gradio的版本，敬請見諒***



---



**Google Sheet**：https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing







In [3]:
# ==============================
# 🔹 收支管理＋Gemini 收支分析（整合版）
# ==============================

# ========== 套件載入與授權 ==========
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import pandas as pd
import datetime
import pytz
import google.generativeai as genai
from google.colab import userdata

# ========== 設定 ==========
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing"
SPREADSHEET_ID = "1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84"

# 開啟 Google Sheet
gsheets = gc.open_by_url(SPREADSHEET_URL)
worksheet1 = gsheets.worksheet("工作表1")
worksheet2 = gsheets.worksheet("工作表2")

# ========== 功能選單 ==========
action = input("請選擇動作（1: 新增資料, 2: 刪除資料, 3: Gemini收支分析）: ")

# ========== 1️⃣ 新增資料 ==========
if action == "1":
    tz = pytz.timezone("Asia/Taipei")
    now = datetime.datetime.now(tz)
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M")

    categorynumber = input("請輸入項目（1: 早餐, 2: 午餐, 3: 晚餐, 4: 飲料, 5: 交通, 6: 就學用品, 7: 日常用品）: ")
    item = input("請輸入品項: ")
    amount = float(input("請輸入金額: "))
    aa = input("是否要AA分攤（yes or no）: ")
    if aa == "yes":
        people = int(input("請輸入分攤人數: "))
        amount = amount // people
        print(f"這筆支出實付 {amount} 元")

    category_dict = {
        "1": "早餐", "2": "午餐", "3": "晚餐", "4": "飲料",
        "5": "交通", "6": "就學用品", "7": "日常用品"
    }
    category = category_dict.get(categorynumber, "其他")

    new_data = pd.DataFrame([{
        '日期': date_str,
        '時間': time_str,
        '項目': category,
        '品項': item,
        '金額': amount
    }])

    worksheet1.append_rows(values=new_data.values.tolist(), value_input_option='USER_ENTERED')
    print("✅ 資料已成功寫入 Google 工作表！")

# ========== 2️⃣ 刪除資料 ==========
elif action == "2":
    all_data = worksheet1.get_all_values()
    df = pd.DataFrame(all_data[1:], columns=all_data[0])

    print("\n目前資料：")
    print(df)

    row_to_delete = int(input("\n請輸入要刪除的列號: "))
    worksheet1.delete_rows(row_to_delete + 2)
    print(f"✅ 第 {row_to_delete} 列號已成功刪除！")

# ========== 3️⃣ Gemini 收支分析 ==========
elif action == "3":
    try:
        print("🔍 Gemini 收支分析中...")

        # ========== 設定 Gemini ==========
        genai.configure(api_key=userdata.get("gemini"))
        model = genai.GenerativeModel("gemini-2.5-flash")

        # ========== 讀取資料 ==========
        all_data = worksheet1.get_all_values()
        if not all_data or len(all_data) < 2:
            print("⚠️ 沒有資料可以分析")
        else:
            df = pd.DataFrame(all_data[1:], columns=all_data[0])

            prompt = f"""
            以下是我的收支表：
            {df.to_string(index=False)}

            請根據資料，給我收支的總結與三點具體建議，
            包含：花費最多的項目、可能的節省方向、以及理財提醒。
            """

            response = model.generate_content(prompt, request_options={"timeout": 60})
            advice = response.text if response.text else "⚠️ Gemini 沒有回傳內容"

            tz = pytz.timezone("Asia/Taipei")
            timestamp = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

            worksheet2.append_row([timestamp, advice])

            print("✅ 已成功將分析結果寫入『工作表2』！")
            print("\n📊 Gemini 建議：\n")
            print(advice)

    except Exception as e:
        print(f"❌ 發生錯誤: {e}")

# ========== 其他輸入錯誤 ==========
else:
    print("輸入錯誤，請輸入 1、2 或 3。")


請選擇動作（1: 新增資料, 2: 刪除資料, 3: Gemini收支分析）: 3
🔍 Gemini 收支分析中...
✅ 已成功將分析結果寫入『工作表2』！

📊 Gemini 建議：

好的，這是一份根據您的收支表所做的總結與建議：

---

### 收支總結

根據您提供的資料，截至 2025-10-01，您的總花費為 **3264 元**。

各項目的花費細項如下：

*   **交通:** 1240 元 (主要為定期票)
*   **就學用品:** 1060 元 (主要為課本和工作服/護目鏡)
*   **午餐:** 330 元
*   **飲料:** 255 元
*   **晚餐:** 254 元
*   **日常用品:** 70 元
*   **早餐:** 55 元

---

### 三點具體建議

#### 1. 花費最多的項目分析

*   **交通費 (1240 元)** 是您本次紀錄中花費最高的項目。其中「定期票」佔了 1200 元，這通常是一筆固定且必要的開銷，如果能充分利用，性價比很高。另外有兩筆 Youbike 費用，這類短期交通費用可以透過更精準的規劃，例如步行短程或利用定期票涵蓋的交通工具來減少。
*   **就學用品 (1060 元)** 排名第二，其中「教概課本 (800 元)」和「工作服與護目鏡 (235 元)」是較大筆的初期投資。這類費用在學期初或特定需求時較為集中，後續應該會降低。

#### 2. 可能的節省方向

檢視您的日常開銷，有幾個方向可以考慮節省：

*   **減少飲料花費 (255 元):** 您的飲料支出相當頻繁且金額不低，在短短兩週多內累積了 255 元。考慮自備水壺、減少購買手搖飲頻率，或選擇更經濟的飲料（如在家泡茶/咖啡）。這會是您最容易看到效果的節省點。
*   **優化餐費支出 (午餐 330 元 + 晚餐 254 元 = 584 元):** 午餐和晚餐是日常開銷的大宗。您可以嘗試：
    *   **自行準備餐點：** 偶爾從家裡帶便當或簡單烹煮，可以顯著降低外食費用。
    *   **選擇更經濟的店家：** 比較周邊餐館價格，尋找性價比更高的選項。
    *   **利用學生優惠：** 如果有學生餐廳或校園周邊提供學生優惠的店家，多加利用。
*   **小額交通費規劃：** 您有兩